In [ ]:
#!/usr/bin/python

import csv
import os
import numpy as np
from torch.utils import data
from torchvision import datasets, transforms
import random
from multiprocessing import cpu_count
import torch
from PIL import Image, ImageDraw #masking
from sklearn.model_selection import train_test_split

def transform_image(image):
    custom_transformer = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((270, 270))])
    image_tr = custom_transformer(image)

    return image_tr

def convert_label(label) :
    if label == 'Wake' : label = 0
    elif label == 'N1' : label = 1
    elif label == 'N2' : label = 2
    elif label == 'N3' : label = 3
    elif label == 'REM' : label = 4

    return label


class CustomDataset(data.Dataset):
    def __init__(self, phase='train', root=''):
        self.root = root
        self.phase = phase
        self.data = {}

        if self.phase == "train" or self.phase == 'validation':
            path = os.path.join(root,'train'+'set-for_user.csv')
        else:
            path = os.path.join(root,self.phase+'set-for_user.csv')

        f = open(path, 'r', encoding='utf-8-sig')
        rdr = csv.reader(f)

        lst_img = []
        lst_label = []

        #if self.phase == 'train':
        #    rdr = rdr[:448000]
        #elif self.phasee == 'validation':
        #    rdr = rdr[448000:]
        #else: rdr = rdr

        for idx, item in enumerate(rdr):
            path_img = os.path.join(self.root, item[0], item[1])
            if self.phase == 'train' or self.phase == 'validation' :
                label = item[2]
                label = convert_label(label)
            else : label = -1

            lst_img.append(path_img)
            lst_label.append(label)

        if self.phase == 'train':
            lst_img = lst_img[:448460]
            lst_label = lst_label[:448460]

        elif self.phase == 'validation':
            lst_img = lst_img[448460:]
            lst_label = lst_label[448460:]
        else:
            lst_img = lst_img
            lst_label = lst_label

        self.data['image'] = lst_img
        self.data['label'] = lst_label


    def __getitem__(self, index):
        path = self.data['image'][index]
        img = Image.open(path)

        # masking
        #im = ImageDraw.Draw(img)
        #im.rectangle(((0,107),(480,204)), fill = (0,0,0),outline=(0, 0,0), width=2)
        #im.rectangle(((0,225),(480,270)), fill = (0,0,0),outline=(0, 0,0), width=2)

        img = transform_image(img)
        label = self.data['label'][index]
        return img, label

    def __len__(self):
        return len(self.data['image'])


def data_loader(phase, path_dataset = '../../../DATA', batch_size = 32, num_workers = 1):


    if phase == 'train'  or phase =='validation':
        train_dataset = CustomDataset(phase='train', root=path_dataset)
        valid_dataset = CustomDataset(phase='validation', root=path_dataset)

        print(len(train_dataset))
        print(len(valid_dataset))

        #train_dataset, test_dataset = train_test_split(dataset, test_size = 0.2, shuffle= False)
        #print(len(train_dataset))
        #train_size = int(0.8 * len(dataset))
        #test_size = len(dataset) - train_size
        #train_dataset, test_dataset = data.random_split(dataset, [train_size, test_size])

        if phase == 'train':
            dataloader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers = num_workers, shuffle=True)
        else:
            dataloader = data.DataLoader(dataset=valid_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    else :
        dataset = CustomDataset(phase=phase, root=path_dataset)
        dataloader = data.DataLoader(dataset=dataset, batch_size=batch_size, num_workers = num_workers, shuffle=False)

    return dataloader


In [ ]:

#!/usr/bin/python

from J_loader3 import data_loader
import csv
import torch
import os

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

#from pytorch_lightning.metrics import F1

def save_model(model, optimizer) :
    model_cpu = model.to('cpu')
    state = {
        'model' : model_cpu.state_dict(),
        'optimizer' : optimizer.state_dict()}
       # 'scheduler' : scheduler.state_dict()
    if not(os.path.isdir('./saved_model')) : os.mkdir('./saved_model')
    torch.save(state, './saved_model/saved_model_270.pth')

class CNN(nn.Module):

    def __init__ (self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
        in_channels = 1, out_channels = 8,
        kernel_size = 7,
        padding = 0)
        self.conv2 = nn.Conv2d(
        in_channels = 8, out_channels = 16,
        kernel_size = 7,
        padding = 1)
        self.conv3 = nn.Conv2d(
        in_channels = 16, out_channels = 16,
        kernel_size = 7,
        padding = 1)
        self.conv4 = nn.Conv2d(
        in_channels = 16, out_channels = 16,
        kernel_size = 5,
        padding = 1)
        self.conv5 = nn.Conv2d(
        in_channels = 16, out_channels = 32,
        kernel_size = 5,
        padding = 1)

        self.pool = nn.MaxPool2d(
        kernel_size = 2,
        stride = 2)
        self.Drop = nn.Dropout(p=0.2)

        self.fc1 = nn.Linear(6 * 6 * 32, 512)
        self.fc2 = nn.Linear(512, 5)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.Drop(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.Drop(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.Drop(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.Drop(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.Drop(x)

        x = x.view(-1, 6 * 6 * 32)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x)

        return x

def train_val():

    # define or import model
    model = CNN().to(DEVICE)
    # define loss fc
    loss_fn = nn.CrossEntropyLoss()
    # define optimizer
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr = 0.001)
    # define scheduler
    #scheduler = StepLR(optimizer, step_size = 5, gamma =0.5)
    print(model)

    train_loader = data_loader(phase='train', batch_size=32)
    valid_loader = data_loader(phase='validation', batch_size=32)

    for (X_train, y_train) in train_loader:
        print(X_train.size(), X_train.type())
        print(y_train.size(), y_train.type())
        break

    epoch = 10
    for Epoch in range(0, epoch) :
        lst_out = []
        lst_label = []

        model.train()
        train_loss = 0
        correct = 0
        for batch_idx, (image, label) in enumerate(train_loader):
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            optimizer.zero_grad()
            ### Example ###
            out = model(image)
            ### Calculate loss, backward loss and optimizer step ###
            loss = loss_fn(out, label)
            train_loss += loss.item()
            prediction = out.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
            #optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_idx % 1000 == 0:
                print("Train Epoch: {} [{}/{}({:.0f}%)]".format(Epoch+1, batch_idx*len(image), len(train_loader.dataset), 100.*batch_idx/len(train_loader)))

        train_loss /= len(train_loader.dataset)
        train_accuracy = 100.*correct/len(train_loader.dataset)


        model.eval()
        valid_loss = 0
        correct = 0
        with torch.no_grad():
            for image, label in valid_loader:
                image = image.to(DEVICE)
                label = label.to(DEVICE)
                output = model(image)
                valid_loss += loss_fn(output, label).item()
                prediction = output.max(1, keepdim = True)[1]
                lst_out += prediction
                lst_label +=label
                correct += prediction.eq(label.view_as(prediction)).sum().item()

        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = 100.*correct/len(valid_loader.dataset)


            ### Sample ###
           # lst_out += [0,1,2,3,4]
           # lst_label += [0,1,2,3,4]
            ###########

        print("\n [EPOCH: {}] \n Train Loss: {:.4f}, \t Train Accuracy: {:.2f}".format(Epoch+1, train_loss, train_accuracy))
        print(" Valid Loss: {:.4f}, \t Valid Accuracy: {:.2f}".format(valid_loss, valid_accuracy))
        #scheduler.step()
    save_model(model, optimizer)

    #f1 = F1(num_classes=5, average = 'macro')
    #print(lst_out, lst_label)
    #print(f1(torch.tensor(lst_out), torch.tensor(lst_label)))

    #print("\n [EPOCH: {}] \n Train Loss: {:.4f}, \t Train Accuracy: {:.2f}".format(Epoch+1, train_loss, train_accuracy))
    #print(" Valid Loss: {:.4f}, \t Valid Accuracy: {:.2f}".format(valid_loss, valid_accuracy))

def test() :
    dic_label = {0:'Wake', 4:'REM', 1:'N1', 2:'N2', 3:'N3'}
    test_loader = data_loader(phase='test', batch_size= 32)
    f1 = open('./test_result_270.csv', 'w', encoding ='utf-8-sig', newline='')
    wr = csv.writer(f1)
    lst_out = []
    #lst_label = []
    ct = 0


    ### Example ###

    #DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # use GPU (CPU는 None으로 설정)
    print(DEVICE)
    # define or import model
    model = CNN()
    # load model
    state = torch.load('./saved_model/saved_model_270.pth')
    model.load_state_dict(state['model'])
    #DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # use GPU (CPU는 None으로 설정)
    #print(DEVICE)
    model = model.to(DEVICE)

    for batch_idx, (image, _) in enumerate(test_loader):
        image = image.to(DEVICE)
        output = model(image)
        prediction = output.max(1, keepdim = True)[1]
        #print(prediction)
        lst_out +=prediction
       # lst_label +=label

        #example
        #lst_out = [0,1,2,3,4]

        #print(lst_out)
        if batch_idx % 1000 == 0:
            print("[{}/{}({:.0f}%)]".format(batch_idx*len(image), len(test_loader.dataset), 100.*batch_idx/len(test_loader)))
    for idx, item in enumerate(lst_out) :
        if idx % 1000 == 0:
            print("[{}/{}({:.0f}%)]".format(idx*len(image), len(test_loader.dataset), 100.*idx/len(test_loader)))

        wr.writerows([[dic_label[item[0].cpu().numpy().item()]]])

if __name__ == "__main__":

    DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # use GPU (CPU는 None으로 설정)
    print(DEVICE)

    train_val()
    test()